In [1]:
import pandas as pd
import datetime
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import plotly.express as px

In [2]:
df = pd.read_csv('data/created/flagged_tweets_cleaned.csv')
len(df)

4261

In [3]:
df['content'] = df['content'].str.strip()

In [4]:
print(len(df))
df = df.groupby('content', as_index=False).first().reset_index(drop=True)
len(df)

4261


4185

In [5]:
docs = list(df['content'])

In [6]:
for i,j in enumerate(docs):
    docs[i] = re.sub(r'http\S+', '', j)

In [7]:
for idx, tweet in enumerate(docs):
    stop = set(stopwords.words('english') + list(string.punctuation))
    words = [i for i in word_tokenize(tweet.lower()) if i not in stop]
    docs[idx] = words

In [8]:
terms = {}
    
for i in docs:
    for word in i:
        try:
            terms[word] += 1
        except KeyError:
            terms[word] = 1

In [9]:
tdf = pd.DataFrame.from_dict(terms, orient='index')

In [10]:
tdf['term'] = tdf.index
tdf = tdf.rename(columns={0:'freq'})
tdf = tdf.reset_index()
tdf = tdf[['term', 'freq']]

In [11]:
tdf = tdf.sort_values('freq', ascending=False).reset_index()
tdf = tdf[['term', 'freq']]

In [12]:
punc = ["“", "n't", "'s", "”", "...", "'m", "’", "nan", "''", "``", "ln", "na"]
tdf = tdf.loc[tdf['term'].isin(punc) == False].reset_index()
tdf = tdf[['term', 'freq']]
tdf = tdf.sort_values('freq', ascending=False).reset_index()
tdf['rank'] = tdf.index + 1
terms = tdf.head(25)

In [13]:
tdf.loc[tdf['term'] == 'trump']


,index,term,freq,rank
87,87,trump,75,88


In [14]:
terms.to_csv('data/created/terms.csv')

In [15]:
fig = px.bar(terms, x='term', y='freq', template='plotly_dark', title="The 25 Most Common Words From Tweets Flagged by Social Sentinel<br><i>Excludes common 'stopwords' like 'the', 'it', 'and' etc.</i>", labels={
                     "term": "<b>Word</b>",
                     "freq": "<b>Frequency</b>"})
fig.update_traces(marker_color='#a3f26b')
fig.show()